### Data Augumentation

In [1]:
import numpy as np
import os, sys, cv2, time
import matplotlib.pyplot as plt
import mxnet as mx
from mxnet import gluon, init, autograd, nd, image
from mxnet.gluon import data as gdata, utils as gutils, nn, loss as gloss
from shutil import copyfile
from mxnet.gluon.data.vision import datasets, transforms
from sklearn.model_selection import train_test_split

C:\Users\lab\AppData\Local\Continuum\anaconda3\envs\mxnet-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
target_folder = 'E:/Guan-Ming/Deep_ML/train_data_resize/'
output_folder = 'E:/Guan-Ming/Deep_ML/train_data_final_transposed/'
test_data_folder = 'E:/Guan-Ming/Deep_ML/test_data_resize/'

In [3]:
# name = 'resized_two_50.jpg'
# img = image.imread(target_folder+name)
# color_aug = gdata.vision.transforms.RandomColorJitter(brightness=0.25, contrast=0.6)
# for i in range(6):
#     name_jitter = str('color_jitter%d_'%(i))
#     img_color_jitter = color_aug(img)
#     img_color_jitter_np = img_color_jitter.asnumpy()
#     plt.imshow(img_color_jitter_np)
#     plt.axis('off')
#     plt.show()
# img.shape

In [7]:
files = os.listdir(target_folder)
pictures_resized = []
for f in files:
    if '.jpg' in f:
        pictures_resized.append(f)

In [8]:
for index in range(len(pictures_resized)):
#for index in range(1):
    
    name = pictures_resized[index]
    img = image.imread(target_folder+name)
    img_np = img.asnumpy()
    #num_jitter = 5
    num_jitter = 3
    #color_aug = gdata.vision.transforms.RandomColorJitter(brightness=0.25, contrast=0.6)
    color_aug = gdata.vision.transforms.RandomColorJitter(brightness=0.4, contrast=0.3)
    aug_1 = gdata.vision.transforms.RandomFlipLeftRight()
    aug_2 = gdata.vision.transforms.RandomFlipTopBottom()

    success = False
    while not success:
        img_lr_flip = aug_1(img)
        img_lr_flip = img_lr_flip.asnumpy()
        if (img_lr_flip!=img_np).sum()!=0:
            success = True
            cv2.imwrite(output_folder+'left_right_flipped'+name, img_lr_flip[:,:,::-1])

    success = False
    while not success:
        img_tb_flip = aug_2(img)
        img_tb_flip = img_tb_flip.asnumpy()
        if (img_tb_flip!=img_np).sum()!=0:
            success = True
            cv2.imwrite(output_folder+'top_down_flipped'+name, img_tb_flip[:,:,::-1])

    for i in range(num_jitter):
        name_jitter = str('color_jitter%d_'%(i))
        img_color_jitter = color_aug(img)
        img_color_jitter_np = img_color_jitter.asnumpy()
        cv2.imwrite(output_folder+name_jitter+name, img_color_jitter_np[:,:,::-1])
        success = False
        while not success:
            img_lr_flip = aug_1(img_color_jitter)
            img_lr_flip = img_lr_flip.asnumpy()
            if (img_lr_flip!=img_color_jitter_np).sum()!=0:
                success = True
                cv2.imwrite(output_folder+'left_right_flipped'+name_jitter+name, img_lr_flip[:,:,::-1])

        success = False
        while not success:
            img_tb_flip = aug_2(img_color_jitter)
            img_tb_flip = img_tb_flip.asnumpy()
            if (img_tb_flip!=img_color_jitter_np).sum()!=0:
                success = True
                cv2.imwrite(output_folder+'top_down_flipped'+name_jitter+name, img_tb_flip[:,:,::-1])
    copyfile(target_folder+name, output_folder+name)

In [9]:
files = os.listdir(output_folder)
pictures_list = []
for f in files:
    if '.jpg' in f:
        pictures_list.append(f)
        
for name in pictures_list:
    img = image.imread(output_folder+name)
    img_np = img.asnumpy()
    
    img_transpose = np.zeros_like(img_np)
    for channels in range(3):
        img_transpose[:,:,channels] = img_np[:,:, channels].T
    cv2.imwrite(output_folder+'transposed_'+name, img_transpose[:,:,::-1])

### Read in training data

In [4]:
files = os.listdir(output_folder)
total_train_data = len(files)
train_data_x = nd.zeros((total_train_data, 256, 256, 3)).astype(np.uint8)
train_data_y = nd.zeros(total_train_data)
total_num = 0

for index, f in enumerate(files):
    train_data_x[index] = image.imread(output_folder+f)
    if 'zero' in f:
        train_data_y[index] = 0
        total_num += 1
    elif 'one' in f:
        train_data_y[index] = 1
        total_num += 1
    elif 'two' in f:
        train_data_y[index] = 2
        total_num += 1
    elif 'three' in f:
        train_data_y[index] = 3
        total_num += 1
    elif 'four' in f:
        train_data_y[index] = 4
        total_num += 1
    elif 'five' in f:
        train_data_y[index] = 5
        total_num += 1
    else:
        raise RuntimeError('Cannot label training data!')

if total_num == total_train_data:
    print("Sum check pass!")

Sum check pass!


In [5]:
# train_mean_r = (train_data_x[:,:,:,0].asnumpy()/255).mean()
# train_mean_g = (train_data_x[:,:,:,1].asnumpy()/255).mean()
# train_mean_b = (train_data_x[:,:,:,2].asnumpy()/255).mean()
# train_std_r = (train_data_x[:,:,:,0].asnumpy()/255).std()
# train_std_g = (train_data_x[:,:,:,1].asnumpy()/255).std()
# train_std_b = (train_data_x[:,:,:,2].asnumpy()/255).std()
# train_mean = [train_mean_r, train_mean_g, train_mean_b]
# train_std = [train_std_r, train_std_g, train_std_b]
train_mean = [0.6033889327634111, 0.5501308484707325, 0.5178991890040533]
train_std = [0.2938110721034864, 0.2913514030355614, 0.2942829946844301]
train_mean, train_std

([0.6033889327634111, 0.5501308484707325, 0.5178991890040533],
 [0.2938110721034864, 0.2913514030355614, 0.2942829946844301])

### Read in testing data

In [6]:
files = os.listdir(test_data_folder)
total_test_data = len(files)
test_data_x = nd.zeros((total_test_data, 256, 256, 3)).astype(np.uint8)
test_data_y = nd.zeros(total_test_data)
total_num = 0
np.random.seed = 123456
test_index_order = np.arange(total_test_data)
np.random.shuffle(test_index_order)

for index, index_shuffled in enumerate(test_index_order):
    f = files[index_shuffled]
    test_data_x[index] = image.imread(test_data_folder+f)
    label = f.replace('resized_','').split('_', 1)[0]
    if label == '0':
        test_data_y[index] = 0
        total_num += 1
    elif label == '1':
        test_data_y[index] = 1
        total_num += 1
    elif label == '2':
        test_data_y[index] = 2
        total_num += 1
    elif label == '3':
        test_data_y[index] = 3
        total_num += 1
    elif label == '4':
        test_data_y[index] = 4
        total_num += 1
    elif label == '5':
        test_data_y[index] = 5
        total_num += 1
    else:
        raise RuntimeError('Cannot label testing data!')

# for index, f in enumerate(files):
#     test_data_x[index] = image.imread(test_data_folder+f)
#     label = f.split('_', 1)[0]
#     if label == '0':
#         test_data_y[index] = 0
#         total_num += 1
#     elif label == '1':
#         test_data_y[index] = 1
#         total_num += 1
#     elif label == '2':
#         test_data_y[index] = 2
#         total_num += 1
#     elif label == '3':
#         test_data_y[index] = 3
#         total_num += 1
#     elif label == '4':
#         test_data_y[index] = 4
#         total_num += 1
#     elif label == '5':
#         test_data_y[index] = 5
#         total_num += 1
#     else:
#         raise RuntimeError('Cannot label training data!')
        
if total_num == total_test_data:
    print("Sum check pass!")

Sum check pass!


### Use gdata to form train dataset and test dataset

In [7]:
batch_size = 64
train_dataset = gdata.ArrayDataset(train_data_x, train_data_y)
test_dataset = gdata.ArrayDataset(test_data_x, test_data_y)

# index = np.random.randint(total_test_data)
# count = 0
# for X, y in test_dataset:
#     if count==index:
#         plt.imshow(X.asnumpy())
#         plt.axis('off')
#         plt.show()
#         print('Label is %d .'%y)
#         break
#     count += 1
    
transformer = []
transformer += [gdata.vision.transforms.ToTensor()] # transer the train data from shape (sample, H, W, channel) to (sample, channel, H, W) and rescale to between 0 and 1 
transformer += [gdata.vision.transforms.Normalize(train_mean, train_std)]
transformer = gdata.vision.transforms.Compose(transformer)
num_workers = 0 if sys.platform.startswith('win32') else 4
train_iter = gdata.DataLoader(train_dataset.transform_first(transformer), batch_size = batch_size, shuffle=True, num_workers=num_workers)
test_iter = gdata.DataLoader(test_dataset.transform_first(transformer), batch_size = batch_size, shuffle=False, num_workers=num_workers)

In [8]:
# index = np.random.randint(batch_size)
# count = 0
# img = nd.zeros((256, 256, 3))
# for X, y in test_iter:
#     X = X*255
#     img[:,:,0] = X[index,0,:,:]
#     img[:,:,1] = X[index,1,:,:]
#     img[:,:,2] = X[index,2,:,:]
#     plt.imshow( ((img.asnumpy())//1).astype(np.uint8) )
#     plt.axis('off')
#     plt.show()
#     print('Label is %d .'%y[index].asscalar())
#     break

In [9]:
def try_gpu():
    """If GPU is available, return mx.gpu(0); else return mx.cpu()."""
    try:
        ctx = mx.gpu()
        _ = nd.array([0], ctx=ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx

def evaluate_accuracy(data_iter, net, ctx=[mx.cpu()]):
    """Evaluate accuracy of a model on the given data set."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n = nd.array([0]), 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            y = y.astype('float32')
            acc_sum += (net(X).argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n

def _get_batch(batch, ctx):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])

def train(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs):
    """Train and evaluate a model with CPU or GPU."""
    print('training on', ctx)
    loss = gloss.SoftmaxCrossEntropyLoss()
    average_time = 0.0
    
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)            
                l = loss(y_hat, y).sum()
            l.backward()
            trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        
        test_acc = evaluate_accuracy(test_iter, net, ctx)
        epoch_time = time.time() - start
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc, epoch_time))
        average_time += epoch_time
    print('Average time per epoch is %.4f s.'%(average_time/num_epochs))

In [10]:
ctx = try_gpu()

In [10]:
class Residual(nn.Block):
    def __init__(self, num_channels, use_1x1conv=False, strides=1, **kwargs):
        super(Residual, self).__init__(**kwargs) 
        self.conv1 = nn.Conv2D(num_channels, kernel_size=3, padding=1, strides=strides) # this layer down-sampling the input
        self.conv2 = nn.Conv2D(num_channels, kernel_size=3, padding=1)  # the layer doesn't down-sampling the input
        
        if use_1x1conv:
            self.conv3 = nn.Conv2D(num_channels, kernel_size=1, strides=strides)
            # to match the dimension of feature map of conv1 + conv2, the strides must be set to be identicla to conv 1
        else: 
            self.conv3 = None 
            
        self.bn1 = nn.BatchNorm() 
        self.bn2 = nn.BatchNorm()
        
    def forward(self, X):
        Y = nd.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y)) 
        if self.conv3:
            X = self.conv3(X)

        return nd.relu(Y + X)
    
def resnet_block(num_channels, num_residuals, first_block=False): 
    blk = nn.Sequential() 
    for i in range(num_residuals): 
        if i == 0 and not first_block:
            #net.add(Residual(num_channels, use_1x1conv=True, strides=2))
            blk.add(Residual(num_channels, use_1x1conv=True, strides=2))
        else:
            #net.add(Residual(num_channels, strides=1))
            blk.add(Residual(num_channels, strides=1))
    return blk

In [22]:
num_of_output_channels = [64, 128, 256, 512]
num_residuals = [3, 4, 6, 3]

net_RESNET = nn.Sequential() 
net_RESNET.add(nn.Conv2D(64, kernel_size=7, strides=2, padding=3),
        nn.BatchNorm(), 
        nn.Activation('relu'),
        nn.MaxPool2D(pool_size=3, strides=2, padding=1))

for i, num in enumerate(num_of_output_channels):
    if i==0:
        net_RESNET.add(resnet_block(num, num_residuals[i], first_block=True))
    else:
        net_RESNET.add(resnet_block(num, num_residuals[i]))
    net_RESNET.add(nn.Dropout(0.3))

net_RESNET.add(nn.GlobalAvgPool2D(), nn.Dense(6))

In [23]:
#net_RESNET.load_parameters('net_RESNET32_DROPOUT_train_data_transposed.params', ctx= ctx)
net_RESNET.initialize(ctx=ctx, init=mx.init.Xavier(rnd_type ='gaussian', factor_type='in', magnitude=2.0))

In [24]:
for X,_ in train_iter:
    net_RESNET.summary(X.as_in_context(ctx))
    break

--------------------------------------------------------------------------------
        Layer (type)                                Output Shape         Param #
               Input                           (64, 3, 256, 256)               0
            Conv2D-1                          (64, 64, 128, 128)            9472
         BatchNorm-2                          (64, 64, 128, 128)             256
        Activation-3                          (64, 64, 128, 128)               0
         MaxPool2D-4                            (64, 64, 64, 64)               0
            Conv2D-5                            (64, 64, 64, 64)           36928
         BatchNorm-6                            (64, 64, 64, 64)             256
            Conv2D-7                            (64, 64, 64, 64)           36928
         BatchNorm-8                            (64, 64, 64, 64)             256
          Residual-9                            (64, 64, 64, 64)               0
           Conv2D-10        

In [27]:
adam_optimizer = mx.optimizer.Adam(learning_rate=1e-4, beta1=0.9, beta2=0.9)
trainer = gluon.Trainer(net_RESNET.collect_params(), optimizer=adam_optimizer)
train(net_RESNET, train_iter, test_iter, batch_size, trainer, ctx, 20)

training on gpu(0)
epoch 1, loss 0.0300, train acc 0.990, test acc 0.690, time 82.3 sec
epoch 2, loss 0.0289, train acc 0.990, test acc 0.637, time 83.3 sec
epoch 3, loss 0.0279, train acc 0.992, test acc 0.687, time 82.6 sec
epoch 4, loss 0.0266, train acc 0.992, test acc 0.663, time 82.6 sec
epoch 5, loss 0.0223, train acc 0.992, test acc 0.720, time 83.3 sec
epoch 6, loss 0.0241, train acc 0.992, test acc 0.667, time 83.1 sec
epoch 7, loss 0.0185, train acc 0.994, test acc 0.707, time 82.5 sec
epoch 8, loss 0.0142, train acc 0.995, test acc 0.680, time 82.7 sec
epoch 9, loss 0.0159, train acc 0.995, test acc 0.677, time 83.4 sec
epoch 10, loss 0.0199, train acc 0.994, test acc 0.727, time 83.1 sec
epoch 11, loss 0.0155, train acc 0.995, test acc 0.647, time 83.2 sec
epoch 12, loss 0.0219, train acc 0.994, test acc 0.677, time 82.5 sec
epoch 13, loss 0.0158, train acc 0.995, test acc 0.600, time 82.8 sec
epoch 14, loss 0.0143, train acc 0.995, test acc 0.700, time 82.3 sec
epoch 15, 

In [28]:
print('Accuracy of model is %.4f %% .'%(100*evaluate_accuracy(test_iter, net_RESNET, ctx)) )

Accuracy of model is 68.0000 % .


In [21]:
#net_RESNET.save_parameters('C:/Users/lab/Jupyter_Notebook/Guan-Ming/Midterm_Project/NN_params/train_test_independent/RESNET18_DROPOUT_train_data_transposed.params')
#===================================================================================
# num_jitter = 3
# color_aug = gdata.vision.transforms.RandomColorJitter(brightness=0.4, contrast=0.3)
#===================================================================================
# num_of_output_channels = [64, 128, 256, 512]
# num_residuals = 2

# net_RESNET = nn.Sequential() 
# net_RESNET.add(nn.Conv2D(64, kernel_size=7, strides=2, padding=3),
#         nn.BatchNorm(), 
#         nn.Activation('relu'),
#         nn.MaxPool2D(pool_size=3, strides=2, padding=1))

# for i, num in enumerate(num_of_output_channels):
#     if i==0:
#         net_RESNET.add(resnet_block(num, num_residuals, first_block=True))
#     else:
#         net_RESNET.add(resnet_block(num, num_residuals))
#     net_RESNET.add(nn.Dropout(0.1))

# net_RESNET.add(nn.GlobalAvgPool2D(), nn.Dense(6))
#===================================================================================

In [18]:
# net_RESNET.save_parameters('C:/Users/lab/Jupyter_Notebook/Guan-Ming/Midterm_Project/NN_params/train_test_independent/RESNET34_DROPOUT_train_data_transposed.params')
#===================================================================================
# num_jitter = 3
# color_aug = gdata.vision.transforms.RandomColorJitter(brightness=0.4, contrast=0.3)
#===================================================================================
# num_of_output_channels = [64, 128, 256, 512]
# num_residuals = [3, 4, 6, 3]

# net_RESNET = nn.Sequential() 
# net_RESNET.add(nn.Conv2D(64, kernel_size=7, strides=2, padding=3),
#         nn.BatchNorm(), 
#         nn.Activation('relu'),
#         nn.MaxPool2D(pool_size=3, strides=2, padding=1))

# for i, num in enumerate(num_of_output_channels):
#     if i==0:
#         net_RESNET.add(resnet_block(num, num_residuals[i], first_block=True))
#     else:
#         net_RESNET.add(resnet_block(num, num_residuals[i]))
#     net_RESNET.add(nn.Dropout(0.3))

# net_RESNET.add(nn.GlobalAvgPool2D(), nn.Dense(6))
#===================================================================================

In [11]:
class Inception(nn.Block): 
    # c1 - c4 are the number of output channels for each layer in the path
    def __init__(self, c1, c2, c3, c4, strides, **kwargs): 
        super(Inception, self).__init__(**kwargs) 
        # Path 1 is a single 1 x 1 convolutional layer 
        self.p1_1 = nn.Conv2D(c1, kernel_size=1, activation='relu') 
        # Path 2 is a 1 x 1 convolutional layer followed by a 3 x 3 
        # convolutional layer 
        #self.p2_1 = nn.Conv2D(c2[0], kernel_size=1, activation='relu')
        #self.p2_2 = nn.Conv2D(c2[1], kernel_size=3, padding=1, activation='relu')
        self.p2_1 = nn.Conv2D(c2, kernel_size=3, padding=1, activation='relu')
        # Path 3 is a 1 x 1 convolutional layer followed by a 5 x 5
        # convolutional layer 
        #self.p3_1 = nn.Conv2D(c3[0], kernel_size=1, activation='relu') 
        #self.p3_2 = nn.Conv2D(c3[1], kernel_size=5, padding=2, activation='relu')
        self.p3_1 = nn.Conv2D(c3, kernel_size=5, padding=2, activation='relu')
        # Path 4 is a 3 x 3 maximum pooling layer followed by a 1 x 1
        # convolutional layer
        self.p4_1 = nn.MaxPool2D(pool_size=3, strides=1, padding=1) 
        self.p4_2 = nn.Conv2D(c4, kernel_size=1, activation='relu')
        
        self.drop = nn.Dropout(0.15)
        self.pool = nn.MaxPool2D(pool_size=3, strides=strides, padding=1)
        
    def forward(self, x):
        p1 = self.p1_1(x) 
        #p2 = self.p2_1(self.p2_1(x))
        #p3 = self.p3_1(self.p3_1(x))
        p2 = self.p2_1(x)
        p3 = self.p3_1(x)
        p4 = self.p4_2(self.p4_1(x)) 
        # Concatenate the outputs on the channel dimension 
        return self.drop(self.pool(nd.concat(p1, p2, p3, p4, dim=1)))

class Residual_Inception(nn.Block):
    def __init__(self, c1, c2, c3, c4, strides=1, **kwargs):
        super(Residual_Inception, self).__init__(**kwargs) 
        self.incept = Inception(c1, c2, c3, c4, strides)
        #num_channels = c1 + c2[1] + c3[1] + c4
        num_channels = c1 + c2 + c3 + c4
        
        self.conv = nn.Conv2D(num_channels, kernel_size=1, strides=strides)
        # to match the dimension of feature map of conv1 + conv2, the strides must be set to be identicla to conv 1
        self.bn = nn.BatchNorm()
        
    def forward(self, X):
        Y = self.bn(self.incept(X))
        X = self.conv(X)

        return nd.relu(X + Y)
    
def resnet_inception_block(num_channels, num_residuals, first_block=False): 
    blk = nn.Sequential() 
    for i in range(num_residuals): 
        if i == 0 and not first_block:
            #net.add(Residual(num_channels, use_1x1conv=True, strides=2))
            blk.add(Residual_Inception(num_channels[0], num_channels[1], num_channels[2], num_channels[3], strides=2))
        else:
            #net.add(Residual(num_channels, strides=1))
            blk.add(Residual_Inception(num_channels[0], num_channels[1], num_channels[2], num_channels[3], strides=1))
    return blk

In [12]:
#num_of_output_channels = [(32, (32,64), (16, 32), 16), (32, (32,64), (16, 32), 16), (32, (32,64), (16, 32), 16)]
num_of_output_channels = [(16, 32, 16, 8), (32, 64, 32, 8)]
num_residuals = [2, 2]

net_RESNET_INCEPT = nn.Sequential() 
net_RESNET_INCEPT.add(nn.Conv2D(64, kernel_size=7, strides=2, padding=3),
                      nn.BatchNorm(), 
                      nn.Activation('relu'),
                      nn.MaxPool2D(pool_size=3, strides=2, padding=1))

for i, num in enumerate(num_of_output_channels):
    if i==0:
        net_RESNET_INCEPT.add(resnet_inception_block(num, num_residuals[i], first_block=True))
    else:
        net_RESNET_INCEPT.add(resnet_inception_block(num, num_residuals[i]))
    #net_RESNET_INCEPT.add(nn.Dropout(0.3))

net_RESNET_INCEPT.add(nn.GlobalAvgPool2D(), nn.Dense(6))

In [13]:
net_RESNET_INCEPT.initialize(ctx=ctx, init=mx.init.Xavier(rnd_type ='gaussian', factor_type='in', magnitude=2.0))

In [14]:
for X,_ in train_iter:
    net_RESNET_INCEPT.summary(X.as_in_context(ctx))
    break

--------------------------------------------------------------------------------
        Layer (type)                                Output Shape         Param #
               Input                           (64, 3, 256, 256)               0
            Conv2D-1                          (64, 64, 128, 128)            9472
         BatchNorm-2                          (64, 64, 128, 128)             256
        Activation-3                          (64, 64, 128, 128)               0
         MaxPool2D-4                            (64, 64, 64, 64)               0
        Activation-5                     <Symbol conv1_relu_fwd>               0
        Activation-6                            (64, 32, 64, 64)               0
            Conv2D-7                            (64, 32, 64, 64)            2080
        Activation-8                     <Symbol conv2_relu_fwd>               0
        Activation-9                            (64, 64, 64, 64)               0
           Conv2D-10        

In [ ]:
adam_optimizer = mx.optimizer.Adam(learning_rate=1e-3, beta1=0.9, beta2=0.9)
trainer = gluon.Trainer(net_RESNET_INCEPT.collect_params(), optimizer=adam_optimizer)
train(net_RESNET_INCEPT, train_iter, test_iter, batch_size, trainer, ctx, 20)

training on gpu(0)
epoch 1, loss 1.6998, train acc 0.296, test acc 0.180, time 87.6 sec
epoch 2, loss 1.4772, train acc 0.410, test acc 0.230, time 84.3 sec


In [98]:
class DenseBlock(nn.Block): 
    def __init__(self, num_convs, num_channels, **kwargs):
        super(DenseBlock, self).__init__(**kwargs) 
        self.net = nn.Sequential() 
        for _ in range(num_convs): 
            self.net.add(conv_block(num_channels))
            
    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            # Concatenate the input and output of each block on the channel 
            # dimension
            X = nd.concat(X, Y, dim=1) 

        return X
    
def transition_block(num_channels):
    blk = nn.Sequential() 
    blk.add(nn.BatchNorm(), 
            nn.Activation('relu'),
            nn.Conv2D(num_channels, kernel_size=1), 
            nn.AvgPool2D(pool_size=2, strides=2))
    return blk

def conv_block(num_channels): 
    blk = nn.Sequential() 
    blk.add(nn.BatchNorm(), 
            nn.Activation('relu'),
            nn.Conv2D(num_channels, kernel_size=3, padding=1))
    return blk

In [28]:
net_DENSENET = nn.Sequential()
net_DENSENET.add(nn.Conv2D(64, kernel_size=7, strides=2, padding=3), 
        nn.BatchNorm(),
        nn.Activation('relu'), 
        nn.MaxPool2D(pool_size=3, strides=2, padding=1))

num_channels, growth_rate = 64, 16
num_convs_in_dense_blocks = [4, 4, 4, 4, 4]

for i, num_convs in enumerate(num_convs_in_dense_blocks):
    net_DENSENET.add(DenseBlock(num_convs, growth_rate))
    net_DENSENET.add(nn.Dropout(0.3))
#     # This is the number of output channels in the previous dense block 
    num_channels += num_convs * growth_rate
    # A transition layer that haves the number of channels is added between
    # the dense blocks 
    if i != len(num_convs_in_dense_blocks) - 1: 
        num_channels //= 2
        net_DENSENET.add(transition_block(num_channels))
                    
net_DENSENET.add(nn.BatchNorm(), 
        nn.Activation('relu'), 
        nn.GlobalAvgPool2D(),
        nn.Dense(6))

net_DENSENET.initialize(ctx=ctx, init=mx.init.Xavier(rnd_type ='gaussian', factor_type='in', magnitude=2.0))

In [29]:
for X,_ in train_iter:
    net_DENSENET.summary(X.as_in_context(ctx))
    break

--------------------------------------------------------------------------------
        Layer (type)                                Output Shape         Param #
               Input                           (64, 3, 256, 256)               0
            Conv2D-1                          (64, 64, 128, 128)            9472
         BatchNorm-2                          (64, 64, 128, 128)             256
        Activation-3                          (64, 64, 128, 128)               0
         MaxPool2D-4                            (64, 64, 64, 64)               0
         BatchNorm-5                            (64, 64, 64, 64)             256
        Activation-6                            (64, 64, 64, 64)               0
            Conv2D-7                            (64, 16, 64, 64)            9232
         BatchNorm-8                            (64, 80, 64, 64)             320
        Activation-9                            (64, 80, 64, 64)               0
           Conv2D-10        

In [30]:
adam_optimizer = mx.optimizer.Adam(learning_rate=5e-4, beta1=0.9, beta2=0.9)
trainer = gluon.Trainer(net_DENSENET.collect_params(), optimizer=adam_optimizer)
train(net_DENSENET, train_iter, test_iter, batch_size, trainer, ctx, 20)

training on gpu(0)
epoch 1, loss 1.5889, train acc 0.334, test acc 0.183, time 69.5 sec
epoch 2, loss 1.1107, train acc 0.552, test acc 0.333, time 69.2 sec
epoch 3, loss 0.7759, train acc 0.697, test acc 0.317, time 69.7 sec
epoch 4, loss 0.5791, train acc 0.781, test acc 0.423, time 69.3 sec
epoch 5, loss 0.4588, train acc 0.829, test acc 0.483, time 69.3 sec
epoch 6, loss 0.3636, train acc 0.869, test acc 0.383, time 69.4 sec
epoch 7, loss 0.3104, train acc 0.888, test acc 0.487, time 69.3 sec
epoch 8, loss 0.2864, train acc 0.892, test acc 0.570, time 69.2 sec
epoch 9, loss 0.2281, train acc 0.916, test acc 0.510, time 69.4 sec
epoch 10, loss 0.2055, train acc 0.926, test acc 0.543, time 69.6 sec
epoch 11, loss 0.1709, train acc 0.939, test acc 0.553, time 69.5 sec
epoch 12, loss 0.1439, train acc 0.947, test acc 0.547, time 69.4 sec


KeyboardInterrupt: 

In [26]:
print('Accuracy of model is %.4f %% .'%(100*evaluate_accuracy(test_iter, net_DENSENET, ctx)))

Accuracy of model is 71.6667 % .


In [27]:
#net_DENSENET.save_parameters('C:/Users/lab/Jupyter_Notebook/Guan-Ming/Midterm_Project/NN_params/train_test_independent/DENSENET_DROPOUT_train_data_transposed.params')
#===================================================================================
# num_jitter = 3
# color_aug = gdata.vision.transforms.RandomColorJitter(brightness=0.4, contrast=0.3)
#===================================================================================
# net_DENSENET = nn.Sequential()
# net_DENSENET.add(nn.Conv2D(64, kernel_size=7, strides=2, padding=3), 
#         nn.BatchNorm(),
#         nn.Activation('relu'), 
#         nn.MaxPool2D(pool_size=3, strides=2, padding=1))

# num_channels, growth_rate = 64, 16
# num_convs_in_dense_blocks = [4, 4, 4, 4, 4]

# for i, num_convs in enumerate(num_convs_in_dense_blocks):
#     net_DENSENET.add(DenseBlock(num_convs, growth_rate))
#     net_DENSENET.add(nn.Dropout(0.15))
#     # This is the number of output channels in the previous dense block 
#     num_channels += num_convs * growth_rate
#     # A transition layer that haves the number of channels is added between
#     # the dense blocks 
#     if i != len(num_convs_in_dense_blocks) - 1: 
#         num_channels //= 2
#         net_DENSENET.add(transition_block(num_channels))
                    
# net_DENSENET.add(nn.BatchNorm(), 
#         nn.Activation('relu'), 
#         nn.GlobalAvgPool2D(),
#         nn.Dense(6))

# net_DENSENET.initialize(ctx=ctx, init=mx.init.Xavier(rnd_type ='gaussian', factor_type='in', magnitude=2.0))
#===================================================================================

In [285]:
mnist_train = gdata.vision.FashionMNIST(root=os.path.join('~', '.mxnet', 'datasets', 'fashion-mnist'), train=True)
mnist_test = gdata.vision.FashionMNIST(root=os.path.join('~', '.mxnet', 'datasets', 'fashion-mnist'), train=False)

In [159]:
for X, _ in mnist_train:
    print(X.shape, X.dtype)
    break

(28, 28, 1) <class 'numpy.uint8'>


In [305]:
mnist_test[:10][1]

array([0, 1, 2, 2, 3, 2, 8, 6, 5, 0])